# <B> Clean up </B>

## AutoReload

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## clean

In [20]:
import os
import time
import boto3
from utils.ssm import parameter_store

In [21]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [6]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))

In [7]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

* endpoint

In [8]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [9]:
clean = clean_up()
sm_client = boto3.client('sagemaker')

## 2.training 
strEndPointName = pm.get_params(key="-".join([strPrefix, "ENDPOINT-NAME-DEPLOY"]))
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

## 4.pipeline
strEndPointName = pm.get_params(key="-".join([strPrefix, "ENDPOINT-NAME"]))
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

IndexError: list index out of range

* pipeline

In [10]:
strPipelineName = pm.get_params(key="-".join([strPrefix, "PIPELINE-NAME"]))
strPipelineName

'ad-ts-PIPELINE'

In [11]:
!aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:us-east-1:419974056037:pipeline/ad-ts-pipeline"
}


* model registry

In [12]:
sagemaker_client = boto3.client('sagemaker')

In [13]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [14]:
DeleteModelGroupName = pm.get_params(key=strPrefix + "MODEL-GROUP-NAME")
empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

IndexError: list index out of range

* parameter store

In [15]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]

total_size = len(listParams)
for idx in range(0, total_size, 10):
    start = idx
    if start + 10 < total_size: end = start + 10
    else: end = total_size
    listSubParams = listParams[start:end]
    pm.delete_param(listSubParams)

  parameters: ['ad-ts-ACCOUNT-ID', 'ad-ts-BUCKET', 'ad-ts-ENDPOINT-NAME', 'ad-ts-KINESIS-ANALYTICS-ROLE-ARN', 'ad-ts-KINESIS-APP-NAME', 'ad-ts-PIPELINE-ARN', 'ad-ts-PIPELINE-NAME', 'ad-ts-PREP-DATA-PATH', 'ad-ts-REGION', 'ad-ts-S3-DATA-PATH'] is deleted successfully
  parameters: ['ad-ts-S3-MODEL-ARTIFACT', 'ad-ts-SAGEMAKER-ROLE-ARN', 'ad-ts-STREAM-ARN-INPUT', 'ad-ts-STREAM-ARN-OUTPUT', 'PREFIX'] is deleted successfully


* code commit, buldand pipeline

In [16]:
# from utils.code_pipeline import code_pipeline_handler
# cph = code_pipeline_handler()

In [16]:
# cph.delete_repository(strRepoName="SM-MLOPS-PIPELINE-DEMO")
# cph.delete_build_project(strCodeBuildPJTName="demo-mlops-build")
# cph.delete_pipeline(strPipeLineName="demo-mlops-code-pipeline")

== DELETE REPO ==
  Repository name [SM-MLOPS-PIPELINE-DEMO] was successfully deleted!!
''
''


* event rule

In [29]:
client_events = boto3.client('events')

In [30]:
response = client_events.remove_targets(
    Rule=pm.get_params(key="-".join([strPrefix, "EVENT-RULE-NAME"])),
    Ids=[pm.get_params(key="-".join([strPrefix, "LAMBDA-PIPELINE-MONITOR"]))],
    Force=True
)
response

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': '98094928-6e20-4168-8017-4bf9bc9bef8f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '98094928-6e20-4168-8017-4bf9bc9bef8f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Thu, 04 May 2023 16:15:30 GMT'},
  'RetryAttempts': 0}}

In [31]:
response = client_events.delete_rule(
    Name=pm.get_params(key="-".join([strPrefix, "EVENT-RULE-NAME"])),
    Force=True
)
response

{'ResponseMetadata': {'RequestId': 'd797e441-01ca-4e36-8b86-7f35bf0a2886',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd797e441-01ca-4e36-8b86-7f35bf0a2886',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 04 May 2023 16:15:30 GMT'},
  'RetryAttempts': 0}}

* Lambda

In [32]:
from utils.lambda_func import lambda_handler

In [33]:
lam_handler = lambda_handler(region_name=strRegionName)

In [34]:
lam_handler.delete_function(pm.get_params(key="-".join([strPrefix, "LAMBDA-PIPELINE-MONITOR"])))

  lambda function: [DJ-SM-IMD-LAMBDA] is deleted successfully
